In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 43.4 MB/s 
     |████████████████████████████████| 9.3 MB 2.3 MB/s 
     |████████████████████████████████| 108 kB 29.7 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [11]:
df_5percent=df[0:2865] 


In [12]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,216,4,1,1,1,...,1,1,3,0,7.130302,0.693089,37248,6.796635,0,1
1,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,8192,0.948043,1,0
2,144,3,4,65535,184,120,4,1,0,1,...,1,1,3,0,6.416420,0.000000,22016,4.236472,1,1
3,144,3,4,65535,184,240,7,1,0,1,...,1,3,4,0,6.173033,1.002599,1791176,1.356752,1,0
4,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.344001,3.567366,175448,6.603840,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,144,3,4,65535,184,224,6,1,0,1,...,1,2,4,0,6.489344,0.000000,92672,7.662573,1,1
2861,144,3,4,65535,184,232,4,0,1,1,...,1,0,4,1,5.885863,0.040781,96256,6.419254,0,1
2862,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,6.350862,0.000000,592288,6.511643,0,0
2863,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,354816,7.199813,0,1


In [13]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1357, 69)
class 1: (1508, 69)


In [14]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [15]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     184.0               1.0   
2      144.0   3.0        4.0     65535.0  184.0     120.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     240.0               7.0   
4      144.0   3.0        4.0     65535.0  184.0     256.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
2860   144.0   3.0        4.0     65535.0  184.0     224.0               6.0   
2861   144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
2862   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
2863    80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
2864   144.0   3.0        4.0     65535.0  184.0     240.0               6.0   

      C

In [16]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.0,...,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2.865000e+03,2865.000000,2865.000000,2865.000000
mean,144.503665,9.901571,8.454799,65125.784991,212.312042,222.261780,4.646422,0.989180,0.367190,1.0,...,0.998953,1.363002,3.283421,0.163700,4.852703,2.500199,7.807410e+05,6.388455,0.538569,0.526353
std,329.428205,374.532359,240.016352,5045.567304,1264.263722,49.245139,1.913714,0.103474,0.482123,0.0,...,0.032348,1.621745,1.158805,0.370068,2.577992,2.677370,4.765027e+06,1.108282,0.498597,0.499392
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.577270,0.000000,6.092800e+04,5.701078,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.134653,1.535390,1.218560e+05,6.400870,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.498317,4.663848,3.072000e+05,7.345481,1.000000,1.000000
max,17739.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.997171,7.999568,1.111193e+08,7.999997,1.000000,1.000000


In [17]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [18]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [19]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [20]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      59.375
0.0      40.625
dtype: float64

In [21]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      55.0
0.0      45.0
dtype: float64

In [22]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [23]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [24]:
circuit(weights_init, data[0])

tensor(0.06252232, requires_grad=True)

In [25]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [26]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [27]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [28]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [29]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.10806673, requires_grad=False), tensor(-0.27490958, requires_grad=False)], Y = -1
X = [tensor(-0.15211129, requires_grad=False), tensor(5.84353276, requires_grad=False)], Y =  1
X = [tensor(-0.08179913, requires_grad=False), tensor(-0.27411865, requires_grad=False)], Y = -1
X = [tensor(-0.00356711, requires_grad=False), tensor(-0.27186068, requires_grad=False)], Y = -1
X = [tensor(-0.07821372, requires_grad=False), tensor(-0.2740459, requires_grad=False)], Y = -1


In [30]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [31]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0826740 | Accuracy: 0.5812500 
New best
Iter:     2 | Cost: 1.5226511 | Accuracy: 0.6000000 
Iter:     3 | Cost: 1.2960510 | Accuracy: 0.6000000 
Iter:     4 | Cost: 1.0433291 | Accuracy: 0.6000000 
Iter:     5 | Cost: 0.9675369 | Accuracy: 0.5750000 
Iter:     6 | Cost: 0.9729217 | Accuracy: 0.5687500 
Iter:     7 | Cost: 0.9905913 | Accuracy: 0.5750000 
Iter:     8 | Cost: 0.9978026 | Accuracy: 0.5750000 
Iter:     9 | Cost: 1.0093847 | Accuracy: 0.4062500 
Iter:    10 | Cost: 1.0151367 | Accuracy: 0.4062500 
Iter:    11 | Cost: 1.0146089 | Accuracy: 0.4437500 
Iter:    12 | Cost: 0.9937062 | Accuracy: 0.4750000 
Iter:    13 | Cost: 0.9486747 | Accuracy: 0.5687500 
Iter:    14 | Cost: 0.9551054 | Accuracy: 0.5625000 
Iter:    15 | Cost: 1.0132999 | Accuracy: 0.5687500 
Iter:    16 | Cost: 1.0610153 | Accuracy: 0.5687500 
Iter:    17 | Cost: 0.9833050 | Accuracy: 0.5625000 
Iter:    18 | Cost: 0.9407163 | Accuracy: 0.5625000 
Iter:    19 | Cost: 0.931068

In [32]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [33]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.3399828505337086, Accuracy: 50.0%


In [34]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,-1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,-1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,-1.0
9,-1.0,1.0


In [35]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         2
         1.0       0.91      0.53      0.67        38

    accuracy                           0.50        40
   macro avg       0.45      0.26      0.33        40
weighted avg       0.86      0.50      0.63        40

0.9090909090909091
0.5263157894736842
0.6666666666666666
0.2631578947368421
